## 3D Volume rendering using Napari
#### By Zhenyu Dong (zdong@caltech.edu)

### Load packages

In [ ]:
# Load packages
import napari
import h5py
import os
import numpy as np
import napari_bbox
from napari_animation import Animation
from utils import add_crosssection_layer, Selectcolormap

### Load data

In [ ]:
# Load data file
save_name = 'Embryo_Eightcell'
file_path = 'Data'
file_name = 'data_eightcell.h5'
hf = h5py.File(os.path.join(file_path,file_name), 'r')
pixel_edge_crop = 20
vol = np.array(hf['RI_3D'])
vol = vol[:,pixel_edge_crop:vol.shape[1]-pixel_edge_crop,pixel_edge_crop:vol.shape[2]-pixel_edge_crop,]

if 'nucleus' in hf:
    nucleus = np.array(hf['nucleus'])
    nucleus = nucleus[:,pixel_edge_crop:nucleus.shape[1]-pixel_edge_crop,pixel_edge_crop:nucleus.shape[2]-pixel_edge_crop,]

### Add 3D rendering (MIP)

In [ ]:
# Open napari and add volume rendering
viewer = napari.Viewer()
viewer.dims.ndisplay = 3

# Adjust the scale between x,y,z axis for visualization, this depends on the pixel size of your data
upsampling_rate = 2  # In case you upsampled your data
vol_scale = [ps/upsampling_rate for ps in [1, 6.5/20, 6.5/20]] #sequence: (z, x, y) axis; unit: microns per pixel

# Set your colormap here, it can be a default colormap in napari, or it can be a customized colormap by yourself
color_map = 'inferno'    # default colormaps.
# color_map = Selectcolormap('jet')  # This customized colormap adds an alpha channel based on the value of the data

# Add 3D volume object (an embryo in this example)
RI_bg = 1.33
viewer.add_image(vol, blending='translucent_no_depth', interpolation3d='nearest', depiction='volume', rendering='mip',
                 opacity=1, gamma=1, contrast_limits=[RI_bg,1.338], scale = vol_scale, colormap=color_map)
viewer.dims.axis_labels = ["z","y","x"]

# Add segmentation mask (nucleus segmentation in this example)
viewer.add_image(nucleus, blending='translucent_no_depth', interpolation3d='nearest', depiction='volume', rendering='mip',
                 opacity=0.18, gamma=1, scale = vol_scale, colormap='gray')

### Add bounding box at volume boundary

In [ ]:
# Create labels data and bounding box
bbox = [0, 0, 0, vol.shape[0]*vol_scale[0]-1, vol.shape[1]*vol_scale[1]-1, vol.shape[2]*vol_scale[2]-1]

# Get vertices from bounding box
all_vertices = np.array([
    [bbox[0], bbox[1], bbox[2]],
    [bbox[0], bbox[1], bbox[5]],
    [bbox[0], bbox[4], bbox[5]],
    [bbox[0], bbox[4], bbox[2]],
    [bbox[3], bbox[1], bbox[2]],
    [bbox[3], bbox[1], bbox[5]],
    [bbox[3], bbox[4], bbox[5]],
    [bbox[3], bbox[4], bbox[2]]
]).squeeze()

# Set width of the box
box_width = 0.35

# Set box color
box_color = "white"

# Add bounding box
bb_layer = napari_bbox.BoundingBoxLayer(ndim=3, edge_color=box_color,edge_width=box_width,name='Bounding Box')
bb_layer.add(all_vertices)
viewer.add_layer(bb_layer)

<BoundingBoxLayer layer 'Bounding Box' at 0x1b980540c10>

### Edit axes and scale bar

In [ ]:
# Edit axes
viewer.axes.colored = True
viewer.axes.labels = True
viewer.axes.arrows = True
viewer.axes.visible = True

# Edit scale bar
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'um' 
viewer.scale_bar.length = 20                # length, in units, of the scale bar; it will calculate automatically if you mentioned 'vol_scale' before
viewer.scale_bar.font_size = 14 
viewer.scale_bar.colored = True  
viewer.scale_bar.color = 'white'            # Scale bar text color
viewer.scale_bar.position = 'bottom_right'  # Scale bar position

### Adjust viewing angles and zoom factor

In [ ]:
# Use this for static view
viewer.camera.angles = (-89.78,-8.72,-96.62)
viewer.camera.zoom = 5.8

### Plot cross sections (X-Y,X-Z,Y-Z views)

In [ ]:
# This function is defined in 'utils.py', please import it at the beginning
# define the box width
section_width = 0.2

# xy section
add_crosssection_layer(viewer,vol,vol_scale,0,106,'#ffaa00',section_width)  # change the colormap inside

# xz section
add_crosssection_layer(viewer,vol,vol_scale,1,320,'#55ffff',section_width)

# yz section
add_crosssection_layer(viewer,vol,vol_scale,2,320,'#00aa00',section_width)

### Save screenshot

In [ ]:
# Save volume rendering screenshot
# napari.utils.nbscreenshot(viewer,canvas_only=True)  # show in jupyter notebook
Volume_rendering = viewer.screenshot(path=f'{save_name}_screenshot.png',canvas_only=True,scale=2) # save as screenshot

### Generate 3D rendering video

In [ ]:
# 3D rotation view animation

# Initialize viewing point
view_angle_init_axis1 = -89.78
view_angle_init_axis2 = -8.72
view_angle_init_axis3 = -96.62

# # Resizes window and tries to get a square canvas:
# viewer.window.resize(1000+300, 1000)

# Set parameters for the video
scale_factor = 1  # scale factor for the final output video size
nb_steps = 5  # number of steps between two target angles

# Instantiates a napari animation object for our viewer
animation = Animation(viewer)

# Reset the camera view
viewer.reset_view()

# Adjust the zooming of the image (Setting zoom <= 3.5 ensures the image remains within the canvas when rotated in three dimensions)
viewer.camera.zoom = 3.3

step = 5
rotate_range = 40

arr = np.concatenate([
    np.arange(0, -rotate_range-step, -step), 
    np.arange(-rotate_range, rotate_range+step, step), 
    np.arange(rotate_range, 0, -step),
    np.array([0]),
])

# Start recording key frames after changing viewer state
for angle in arr:
    viewer.camera.angles = (view_angle_init_axis1, view_angle_init_axis2+angle, view_angle_init_axis3)
    animation.capture_keyframe(steps=nb_steps)

for angle in arr:
    viewer.camera.angles = (view_angle_init_axis1, view_angle_init_axis2, view_angle_init_axis3-angle)
    animation.capture_keyframe(steps=nb_steps)

# Render animation as .mp4 video
animation.animate(f'{save_name}_demo.mp4', canvas_only=True, fps=30, scale_factor=scale_factor)

Rendering frames...


100%|██████████| 346/346 [00:10<00:00, 34.09it/s]


#### Note: The rendering video can also be generated manually, by clicking at 'Plugins/wizard (napari-animation)' and capture frame by frame

### Exit napari

In [ ]:
# Close napari
viewer.close()